In [1]:
import tidy3d as td
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [3]:
%matplotlib widget
import hyperspy.api as hs

In [2]:
import os
api_key = os.environ.get("TIDY3D_API_KEY")
import tidy3d.web as web
# web.configure("_blank_")
web.configure(api_key)

Configured successfully.


In [3]:
web.test()

14:37:02 UTC Authentication configured successfully!

# Load data

In [4]:
# pitch and radius sweeps Version 1
pn = 11
rn = 9
t1n = 9
t2n = 9
pitch = np.linspace(50, 300, 11)
radius = np.linspace(20, 60, 9)
thick_1 = np.linspace(100, 300, 9)
thick_2 = np.linspace(10, 100, 9)

# Si_pillar_pitch_50_300 V2
## radius, height = 22.5, 60
ptags = ['sw-d8350829-ff5a-4310-940b-bb5c4174a6480', # pitch = 0.050
         'sw-58b1ecf8-83a9-45a7-9de5-bcf798fb58971', 
         'sw-45b63652-cf97-4413-a437-980a17e5b01e2', # 0.100
         'sw-9e8dc25c-afb7-4a86-904b-1f6d04eebc913', 
         'sw-66478789-03b3-420c-9494-8bd107fe8d914', #0.150
         'sw-4577ec48-7c9e-4d17-898d-fa2ededd5d6d5', 
         'sw-4481fc48-e486-400e-8245-1c5fb97c270d6', # 0.200
         'sw-10c10109-4730-4659-bbf8-ce89669bbcfe7', 
         'sw-53cf6c2d-0f4d-4c39-bc4f-5daa6c9ef0488', #0.250
         'sw-c89365e4-bd43-4aa2-a7ee-b3f95fb3e06d9', 
         'sw-78adbb4f-debc-41a9-9a82-23baaa385e0010'] # 0.300

rtags = ['sw-c624edec-69d9-4144-a4e2-db9e95547a010', # 20
         'sw-c7e3a451-ade7-42f1-b09b-1dde9f9f1d671', 
         'sw-b916af5b-5aeb-4804-9822-d82dd2aa2b502', # 30
         'sw-bbbbf021-96d0-4784-b29a-b9e2206a1e753', 
         'sw-68401dd8-cb74-4784-93da-4d7826d99f834', # 40
         'sw-884c2535-b15a-4a38-a896-9815e68937275', 
         'sw-e940798d-3e9b-48c2-8c82-c5da6020c8516', # 50
         'sw-dbb62fb9-297a-4104-be01-212b275d11cb7', 
         'sw-31f0a9dd-c29d-447e-af3c-957f99ea56138'] # 60

t1tags = ['sw-147665da-6036-4a68-b488-9d698c40f5af0', 
          'sw-e2633443-1542-4759-9f5c-7fef423597671', 
          'sw-5b2595b5-3d89-455d-ad4e-263412606eda2', 
          'sw-868e2495-f976-4a69-99b8-6187910a2c1a3', 
          'sw-d5650155-f436-4ef9-bdc3-5e270201ad7e4', 
          'sw-7f986c05-ad2a-45bd-b4a2-49d2c12b41105', 
          'sw-e0e2c94a-7dc9-4491-86fe-a595c76d658a6', 
          'sw-fb89152f-ba7e-4f7a-baf8-cf001bc6fd557', 
          'sw-858ce4ea-8cff-4ed4-baa8-01b625e4a1458']

t2tags = ['sw-8524baa6-4567-4854-964b-e28b81b4876c0', 
          'sw-5550060a-7ea2-4043-8796-8ecc584676b01', 
          'sw-6b9cec7e-017c-4bf9-ba01-a2f315bcd2062', 
          'sw-f6f16b1f-cdee-4bf3-8097-1da0d1a7c7693', 
          'sw-a91dc687-e93b-4b12-9064-a1f18f0cd8564', 
          'sw-3969a3b1-49fb-4a92-a3e9-85e1b0095a5b5', 
          'sw-a1c1036a-d23e-4d58-a5f4-373882baeccd6', 
          'sw-ee219276-c22f-4871-86dc-dbf52a000d2b7', 
          'sw-ef2b49f3-9ff5-4ccf-8d3d-11605f1fed578', 
          'sw-d02abdba-0aac-445f-8e5e-2c3ef1a2c96f9']

In [85]:
save_dir = '/app/local_project/siHoles_offset'
os.makedirs(save_dir, exist_ok=True)

pol, thickness = 'dipolEx', '25'
name = 'Hxyz'
name_dir = f'miniMon_{pol}_L{thickness}_{name}'

sim_data        = td.SimulationData.from_file(f'{save_dir}/{name_dir}.hdf5')
sim0_ref_data   = td.SimulationData.from_file(f'{save_dir}/{name_dir}_empty.hdf5')

In [15]:
for monitor in sim_data.simulation.monitors:
    print(monitor.name, monitor.type)

DFT_in_plane_slice0 FieldMonitor
DFT_in_plane_slice1 FieldMonitor
DFT_in_plane_slice2 FieldMonitor
DFT_in_plane_slice3 FieldMonitor
DFT_in_plane_slice4.5 FieldMonitor
DFT_in_plane_bottom FieldMonitor
DFT_out_plane FieldMonitor
DFT_in_plane_centre FieldMonitor


In [75]:
monitors = [
# 'DFT_in_plane_slice0',
# 'DFT_in_plane_slice1',
# 'DFT_in_plane_slice2',
# 'DFT_in_plane_slice3',
# 'DFT_in_plane_slice4.5',

'DFT_out_plane',
'DFT_in_plane_centre',
'DFT_in_plane_bottom',
'DFT_out_plane_OFFSET',
'DFT_in_plane_centre_OFFSET',
'DFT_in_plane_bottom_OFFSET',
]

# Checking and definitions

In [16]:
sim_data.simulation.plot_3d()

In [10]:
sim0_ref_data.simulation.plot_3d()

In [9]:
# print(sim_data_D30L25.log)

In [66]:
# sim_data0.simulation.plot_3d()

## Function definitions

In [76]:
h_planck = 6.62607015e-34
e_charge = 1.602176634e-19
def hz_to_ev(f): return (h_planck * f) / e_charge
def ev_to_hz(E): return (E * e_charge) / h_planck

In [77]:
def sum_except_last(arr: np.ndarray):
    """Sum array over all axes except the last one (usually frequency)."""
    return arr if arr.ndim == 0 else arr.sum(axis=tuple(range(arr.ndim - 1)))

# function sum_except_last:
#     Inputs any N-dimensional array, eg.
#         1D: (freq,)
#         2D: (x, freq)
#         3D: (x, y, freq)
#         4D: (x, y, z, freq)
#     sums over all axes except the last one (freq is always preserved)

In [78]:
def compute_normalized_spectrum(sim_data, sim_data0, monitor: str, field: str,
                                hot_pixel_factor=100, eps=1e-18):

    data = getattr(sim_data[monitor], field).values
    ref  = getattr(sim_data0[monitor], field).values

    # absolute difference per spatial point and frequency
    diff = np.abs(data - ref)  # can be used for point monitor too

    # Leaves frequency (last axis) intact:
    diff_total = diff.sum(axis=-1)  # shape (nx,ny) or () for purely point monitors
    
    # Mask to remove hot pixel outliers (anything 100x above median)
    mask = diff_total < (hot_pixel_factor * np.median(diff_total))   
    diff_filtered = diff[mask]
    
    """
    sums over spatial coordinates first (for each frequency)
    then normalise by dividing structure/ empty
    """
    
    # Numerator: sum over filtered points (leave frequency axis)
    numerator = sum_except_last(diff_filtered)             

    # Denominator: sum over all spatial points of reference 
    denominator = sum_except_last(np.abs(ref))

    # normalize (avoid division by zero)
    spec_norm = numerator / (denominator + eps)

    # energy axis in eV
    freqs_hz = getattr(sim_data[monitor], field).f.values
    freqs_ev = hz_to_ev(freqs_hz)
    
    return freqs_ev, spec_norm, numerator, denominator 

In [70]:
# V2 not using
def compute_normalized_spectrum_V2(sim_data, sim_data0, monitor: str, field: str,
                                   hot_pixel_factor=100, eps=1e-18):
    """
    Subtract point-by-point, then sum.
    """
    data = getattr(sim_data[monitor], field).values
    ref  = getattr(sim_data0[monitor], field).values
    
    # Point-wise difference (keeping all dimensions)
    diff = np.abs(data - ref)
    
    # Filter hot pixels based on total difference per spatial point
    diff_total = diff.sum(axis=-1)  # sum over frequency for each point
    mask = diff_total < (hot_pixel_factor * np.median(diff_total))
    diff_filtered = diff[mask]
    
    # Sum over spatial coordinates (leaves frequency axis)
    numerator = sum_except_last(diff_filtered)
    denominator = sum_except_last(np.abs(ref))
    
    spec_norm = numerator / (denominator + eps)
    
    freqs_hz = getattr(sim_data[monitor], field).f.values
    freqs_ev = hz_to_ev(freqs_hz)
    
    return freqs_ev, spec_norm, numerator, denominator   

def compute_spectrum_simplified(sim_data, sim_data0, monitor: str, field: str):
    """With normalization enabled, fields are already source-normalized."""
    data = np.abs(getattr(sim_data[monitor], field).values)
    ref  = np.abs(getattr(sim_data0[monitor], field).values)
    
    # Difference and sum
    diff = data - ref
    spectrum = diff.sum(axis=tuple(range(diff.ndim - 1)))  # sum spatial, keep freq
    
    freqs_hz = getattr(sim_data[monitor], field).f.values
    return hz_to_ev(freqs_hz), spectrum    

In [83]:
# Save_as_csv for all monitors/fields spectra (with compute_normalized_spectrum())
def save_all_spectra(sim_data, sim_data0, monitors, fields=('Ex','Ey','Ez','Hx','Hy','Hz',), out_dir='data/spectra'):
    os.makedirs(out_dir, exist_ok=True)
    rows = []
    
    for mon in monitors:
        for field in fields:
            freqs_ev, spec_norm, num, den = compute_normalized_spectrum(sim_data, sim_data0, mon, field) 
            
            # per-monitor-field CSV
            df = pd.DataFrame({'eV': freqs_ev, 'spectrum': spec_norm})
            fname = os.path.join(out_dir, f"{field}_{mon}_spectrum.csv")
            df.to_csv(fname, index=False)
            
            # also store stacked rows for a combined CSV
            for ev, val in zip(freqs_ev, spec_norm):
                rows.append({'monitor': mon, 'field': field, 'eV': ev, 'spectrum': val})
                
    combined = pd.DataFrame(rows)
    combined.to_csv(os.path.join(out_dir, "all_spectra_combined.csv"), index=False)
    
    print(f"FInished saving individual CSVs to {out_dir} and combined CSV.")

# Generate all fields normalised .csv

In [87]:
# ##### ============ #####   Generate csv files ##### =========================
 
save_all_spectra(sim_data, sim0_ref_data, monitors, fields=('Ex','Ez','Ey'), out_dir=f'data_{name_dir}/spectra')
save_all_spectra(sim_data, sim0_ref_data, monitors, fields=('Hx','Hz','Hy'), out_dir=f'data_{name_dir}/spectra')

FInished saving individual CSVs to data_miniMon_dipolEx_L25_Hxyz/spectra and combined CSV.
FInished saving individual CSVs to data_miniMon_dipolEx_L25_Hxyz/spectra and combined CSV.


# Peak fitting

In [24]:
from scipy.signal import find_peaks, peak_widths
from scipy.optimize import curve_fit
from scipy.special import wofz # for voight

# Gaussian: amplitude A (height), center x0, sigma (std dev)
def gaussian_single(x, A, x0, sigma):
    return A * np.exp(-((x - x0)**2) / (2 * sigma**2))

# Lorentzian: amplitude A (height), center x0, gamma (HWHM)
def lorentzian_single(x, A, x0, gamma):
    return A * (gamma**2) / ((x - x0)**2 + gamma**2)

# Voigt via real(wofz) normalized kernel, scaled by A
def voigt_profile_norm(x, sigma, gamma):
    # normalized Voigt profile (area normalized)
    z = (x + 1j * gamma) / (sigma * np.sqrt(2))
    return np.real(wofz(z)) / (sigma * np.sqrt(2 * np.pi))

def voigt_single(x, A, x0, sigma, gamma):
    return A * voigt_profile_norm(x - x0, sigma, gamma)


In [25]:
# Initial guesses
# Find peaks in `spectrum` (1D). Returns peak indices and widths (FWHM estimate).

def detect_peaks_on_spectrum(freqs_ev, spectrum, rel_prominence=0.05, min_distance_ev=None):
    """
    default = rel_prominence: uses fraction of max(spectrum)
    """
    prominence = rel_prominence * np.nanmax(spectrum)
    
    # convert min_distance_ev to samples if provided
    if min_distance_ev is not None:
        delta = np.abs(freqs_ev[1] - freqs_ev[0])
        min_distance_samples = max(1, int(min_distance_ev / delta))
    else:
        min_distance_samples = None

    peaks, prop = find_peaks(spectrum, prominence=prominence, distance=min_distance_samples)
    
    # widths in number of samples at half prominence; convert to eV
    widths_res = peak_widths(spectrum, peaks, rel_height=0.5)
    widths_samples = widths_res[0]
    delta = np.abs(freqs_ev[1] - freqs_ev[0])
    widths_ev = widths_samples * delta

    results = []
    # # Calculate peak area using trapezoidal integration
    # # Use the detected width to define integration bounds
    # half_width_samples = int(widths_samples[i] / 2)
    # left_idx = max(0, p - half_width_samples)
    # right_idx = min(len(spectrum) - 1, p + half_width_samples)
    
    # # Subtract baseline (minimum value in the range)
    # baseline = np.min(spectrum[left_idx:right_idx+1])
    # peak_area = np.trapz(spectrum[left_idx:right_idx+1] - baseline, 
    #                     freqs_ev[left_idx:right_idx+1])
    for i, p in enumerate(peaks):
        results.append({
            'peak_index': int(p),
            'peak_eV': float(freqs_ev[p]),
            'peak_height': float(spectrum[p]),
            'width_eV': float(widths_ev[i]),
            'prominence': float(prop['prominences'][i]),
            # 'area': float(peak_area)
        })
    
    return results

In [26]:
def fit_single_peak_local(freqs, spectrum, peak_eV, window_half_width_eV=10,
                          model='lorentzian',min_points=10):
    """
    Fit a single peak centered near peak_eV using a local window.
    model in ['lorentzian', 'gaussian', 'voigt'].
    Returns fit parameters and the fitted curve on the local x grid.
    """
    # # select window
    # mask = (freqs >= (peak_eV - window_half_width_eV)) & (freqs <= (peak_eV + window_half_width_eV))
    # x = freqs[mask]
    # y = spectrum[mask]
    # if len(x) < 5:
    #     return None  # not enough points to fit

    # Start with the requested window
    current_window = window_half_width_eV
    
    # Try expanding the window until we have enough points
    for attempt in range(5):
        mask = (freqs >= (peak_eV - current_window)) & (freqs <= (peak_eV + current_window))
        x = freqs[mask]
        y = spectrum[mask]
        if len(x) >= min_points:
            break
        current_window *= 1.5
        
    if len(x) < min_points:
        return {'error': f'Not enough points: {len(x)}, need {min_points}'}

    # initial guesses
    A0 = np.max(y)
    x0 = peak_eV
    dx = x[1] - x[0]
    sigma0 = max( (window_half_width_eV / 4), dx*2 )   # crude
    gamma0 = sigma0

    try:
        if model == 'gaussian':
            p0 = [A0, x0, sigma0]
            popt, pcov = curve_fit(gaussian_single, x, y, p0=p0, maxfev=20000)
            A_fit, x0_fit, sigma_fit = popt
            # FWHM for Gaussian
            fwhm = 2.0 * np.sqrt(2*np.log(2)) * sigma_fit
            fitted = gaussian_single(x, *popt)
            params = {'A':A_fit, 'x0':x0_fit, 'sigma':sigma_fit, 'FWHM':fwhm}

        elif model == 'lorentzian':
            p0 = [A0, x0, gamma0]
            popt, pcov = curve_fit(lorentzian_single, x, y, p0=p0, maxfev=20000)
            A_fit, x0_fit, gamma_fit = popt
            fwhm = 2.0 * gamma_fit
            fitted = lorentzian_single(x, *popt)
            params = {'A':A_fit, 'x0':x0_fit, 'gamma':gamma_fit, 'FWHM':fwhm}

        elif model == 'voigt':
            p0 = [A0, x0, sigma0, gamma0]
            popt, pcov = curve_fit(voigt_single, x, y, p0=p0, maxfev=20000)
            A_fit, x0_fit, sigma_fit, gamma_fit = popt
            
            # approximate FWHM for Voigt (Olivero and Longbothum approximation)
            fwhm = 0.5346 * (2*gamma_fit) + np.sqrt(0.2166 * (2*gamma_fit)**2 + (2.35482 * sigma_fit)**2)
            fitted = voigt_single(x, *popt)
            params = {'A':A_fit, 'x0':x0_fit, 'sigma':sigma_fit, 'gamma':gamma_fit, 'FWHM':fwhm}

        else:
            raise ValueError("model must be 'lorentzian', 'gaussian' or 'voigt'")

        # compute residual sum of squares
        rss = np.sum((y - fitted)**2)
        params.update({'rss':float(rss)})
        return {'x':x, 'y':y, 'fitted':fitted, 'params':params}

    except Exception as e:
        # fitting failed
        return {'error': str(e)}


In [27]:
def fit_and_save_all(sim_data, sim_data0, monitors, fields=('Ex','Ez'),
                     out_dir='data/peak_fits', window_half_width_eV=10, n_peaks_max=6):
    import pandas as pd
    os.makedirs(out_dir, exist_ok=True)
    all_fit_rows = []
    for mon in monitors:
        for field in fields:
            freqs_ev, spec_norm, _, _ = compute_normalized_spectrum(sim_data, sim_data0, mon, field)
            # detect peaks
            peaks = detect_peaks_on_spectrum(freqs_ev, spec_norm, rel_prominence=0.05)
            # optionally limit number of peaks
            if len(peaks) > n_peaks_max:
                peaks = sorted(peaks, key=lambda r: r['peak_height'], reverse=True)[:n_peaks_max]

            # for each peak try the three models and save results
            for pk in peaks:
                center = pk['peak_eV']
                for model in ['lorentzian', 'gaussian', 'voigt']:
                    res = fit_single_peak_local(freqs_ev, spec_norm, center,
                                                window_half_width_eV=window_half_width_eV,
                                                model=model)
                    row = {
                        'monitor': mon, 'field': field, 'model': model,
                        'initial_peak_eV': center
                    }
                    if res is None:
                        row.update({'fit_status': 'not_enough_points'})
                    elif 'error' in res:
                        row.update({'fit_status': 'error', 'error': res['error']})
                    else:
                        p = res['params']
                        row.update({
                            'fit_status': 'ok',
                            'fitted_center_eV': p['x0'],
                            'height': p['A'],
                            'FWHM_eV': p['FWHM'],
                            'rss': p['rss']
                        })
                        # Save fitted curve for this peak to a small CSV
                        df_fit = pd.DataFrame({'eV': res['x'], 'data': res['y'], 'fit': res['fitted']})
                        fname = os.path.join(out_dir, f"{field}_peak_{center:.4f}eV_{mon}_{model}.csv")
                        df_fit.to_csv(fname, index=False)
                    all_fit_rows.append(row)

    df_all = pd.DataFrame(all_fit_rows)
    df_all.to_csv(os.path.join(out_dir, "all_peak_fits_summary.csv"), index=False)
    print("Saved peak fit results to", out_dir)


In [88]:
# ##### ============ #####   Saving peak-detailed data ##### =========================

fit_and_save_all(sim_data, sim0_ref_data, monitors, fields=('Ex','Ez','Ey', 'Hx','Hz','Hy'),
                     out_dir=f'data_{name_dir}/peak_fits', window_half_width_eV=0.2, n_peaks_max=5)

Saved peak fit results to data_miniMon_dipolEx_L25_Hxyz/peak_fits
